In [2]:
import pandas as pd


In [3]:
#Lectura del archivo con la informacion a trabajar obtenida de Kaggle
df = pd.read_csv("../data/processed/df_final.csv")
df

,Venue,Result,Opponent,xG,xGA,Referee,Team
0,Away,0,Chelsea,1.3,1.4,Anthony Taylor,Liverpool
1,Home,1,Bournemouth,3.0,1.3,Thomas Bramall,Liverpool
2,Away,1,Newcastle Utd,0.9,2.0,John Brooks,Liverpool
3,Home,1,Aston Villa,2.5,0.7,Simon Hooper,Liverpool
4,Away,1,Wolves,2.5,0.6,Michael Oliver,Liverpool
...,...,...,...,...,...,...,...
507,Away,2,Crystal Palace,0.8,1.1,Tony Harrington,SheffieldUnited
508,Home,2,Aston Villa,1.4,2.2,Paul Tierney,SheffieldUnited
509,Away,1,Luton Town,1.7,1.8,Chris Kavanagh,SheffieldUnited
510,Home,2,Brighton,0.6,2.9,Stuart Attwell,SheffieldUnited


In [4]:
X = df.drop(columns=["Result"])
y = df['Result']

In [5]:
X.head()

,Venue,Opponent,xG,xGA,Referee,Team
0,Away,Chelsea,1.3,1.4,Anthony Taylor,Liverpool
1,Home,Bournemouth,3.0,1.3,Thomas Bramall,Liverpool
2,Away,Newcastle Utd,0.9,2.0,John Brooks,Liverpool
3,Home,Aston Villa,2.5,0.7,Simon Hooper,Liverpool
4,Away,Wolves,2.5,0.6,Michael Oliver,Liverpool


In [6]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
X= encoder.fit_transform(X)

In [13]:
X



array([[ 0.,  6., 13., 14.,  1., 10.],
       [ 1.,  2., 30., 13., 23., 10.],
       [ 0., 14.,  9., 20.,  9., 10.],
       ...,
       [ 0., 11., 17., 18.,  2., 16.],
       [ 1.,  4.,  6., 29., 22., 16.],
       [ 0., 19.,  6., 12.,  4., 16.]])

In [10]:
# Guardar label en pickle
import pickle 

with open('x_encode.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [1]:
import dagshub
import mlflow

dagshub.init("PROYECTO-CD-Premiere-league", "SantiagoAguirre2002", mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow')

Initialized MLflow to track repo "SantiagoAguirre2002/PROYECTO-CD-Premiere-league"

Repository SantiagoAguirre2002/PROYECTO-CD-Premiere-league initialized!

In [13]:
 #pip install mlflow
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


C:\Users\tatob\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Experimento1

In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


## Experimento 2

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

mlp_clf = MLPClassifier(random_state=42)

grid_search = GridSearchCV(estimator=mlp_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


In [ ]:
import mlflow
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "MLPClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    "hidden_layer_sizes": (50, 50),
    "activation": "logistic",
    "solver": "adam",
    "alpha": 0.0001,
    "learning_rate": "constant",
    "max_iter": 1000,  # Aumentamos el número de iteraciones
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de red neuronal
    mlp_clf = MLPClassifier(**best_params)
    mlp_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = mlp_clf.predict(X_test)
    predictions_proba = mlp_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(mlp_clf, "mlp_classifier")


### Experimento 3


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],  
    
    'max_features': ['auto', 'sqrt'],  
}

rf_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

In [ ]:
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "RandomForestClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    'max_depth': None,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 200,
    'random_state': 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de Random Forest
    rf_clf = RandomForestClassifier(**best_params)
    rf_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = rf_clf.predict(X_test)
    predictions_proba = rf_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(rf_clf, "random_forest_classifier")

# Experimento 4



In [9]:
import mlflow
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define el nombre del modelo y del desarrollador
model_name = "SVM"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Define los parámetros del modelo
svm_params = {"kernel": "linear"}

# Inicia un nuevo experimento en MLflow
with mlflow.start_run(run_name=model_name):
    mlflow.log_param("model", model_name)
    mlflow.log_param("developer", developer)
    mlflow.log_params(svm_params)

    # Entrena el modelo SVM
    svm_model = SVC(**svm_params)
    svm_model.fit(X_train, y_train)

    # Realiza predicciones
    svm_predictions = svm_model.predict(X_test)

    # Calcula métricas
    accuracy = accuracy_score(y_test, svm_predictions)
    precision = precision_score(y_test, svm_predictions, average='weighted')
    recall = recall_score(y_test, svm_predictions, average='weighted')

    # Registra métricas
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)

    # Registra el modelo entrenado
    mlflow.sklearn.log_model(svm_model, "svm_model")

# Experimento 5

In [14]:
import mlflow.tensorflow
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# Define el nombre del modelo y del desarrollador

model_name = "Red Neuronal_2"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Define los parámetros del modelo
red_neuronal_params = {
    "input_dim": X_train.shape[1],  # Dimensión de entrada
    "hidden_layers": [64, 32],       # Número y tamaño de las capas ocultas
    "output_classes": len(set(y_train))  # Número de clases de salida
}

# Inicia un nuevo experimento en MLflow
with mlflow.start_run(run_name=model_name):
    mlflow.log_param("model", model_name)
    mlflow.log_param("developer", developer)
    mlflow.log_params(red_neuronal_params)

    # Define el modelo de red neuronal
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_dim=red_neuronal_params["input_dim"]),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(red_neuronal_params["output_classes"], activation='softmax')
    ])

    # Compila el modelo
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Entrena el modelo
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

    # Realiza predicciones
    predictions = model.predict(X_test)
    y_pred = np.argmax(predictions, axis=1)

    # Calcula métricas
    report = classification_report(y_test, y_pred, output_dict=True)
    confusion = confusion_matrix(y_test, y_pred)

    # Registra métricas
    mlflow.log_metric("accuracy", report['accuracy'])
    mlflow.log_metric("macro_f1_score", report['macro avg']['f1-score'])
    mlflow.log_metric("weighted_precision", report['weighted avg']['precision'])
    mlflow.log_metric("weighted_recall", report['weighted avg']['recall'])
    mlflow.log_metric("weighted_f1_score", report['weighted avg']['f1-score'])

    # Guardar la matriz de confusión en un archivo de texto
    np.savetxt("confusion_matrix.txt", confusion, fmt="%d")

    # Registra la matriz de confusión como un artefacto
    mlflow.log_artifact("confusion_matrix.txt")

Epoch 1/100


C:\Users\tatob\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3597 - loss: 2.8545 - val_accuracy: 0.4146 - val_loss: 1.8170
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5348 - loss: 1.4883 - val_accuracy: 0.4878 - val_loss: 1.0617
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5065 - loss: 0.9897 - val_accuracy: 0.5976 - val_loss: 1.0661
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6205 - loss: 0.9377 - val_accuracy: 0.5854 - val_loss: 1.0519
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6155 - loss: 0.8348 - val_accuracy: 0.5244 - val_loss: 1.0317
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6527 - loss: 0.7858 - val_accuracy: 0.5610 - val_loss: 1.0323
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6610 - loss: 0.7878 - val_accuracy: 0.5488 - val_loss: 0.9912
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6027 - loss: 0.8098 - val_accuracy: 0.5732 - val_loss: 1.

# Experimetno 6

In [16]:
import mlflow.tensorflow
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

# Define una función para crear y compilar el modelo
def create_model(hidden_layer_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(hidden_layer_size, activation='relu', input_dim=X_train.shape[1]),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(len(set(y_train)), activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Define una función de puntuación personalizada
def custom_scorer(model, X, y_true):
    y_pred = model.predict(X)
    y_pred = np.argmax(y_pred, axis=1)
    report = classification_report(y_true, y_pred, output_dict=True)
    return report['weighted avg']['f1-score']

# Define los parámetros a probar en el grid search
param_grid = {'hidden_layer_size': [32, 64, 128]}

# Inicia un nuevo experimento en MLflow
with mlflow.start_run(run_name="Grid Search"):
    # Inicializa el modelo de red neuronal
    model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model)

    # Realiza el grid search
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring=custom_scorer)
    grid_result = grid_search.fit(X_train, y_train)

    # Obtiene los mejores hiperparámetros y resultados
    best_params = grid_result.best_params_
    best_score = grid_result.best_score_

    # Entrena el modelo con los mejores hiperparámetros
    best_model = create_model(best_params['hidden_layer_size'])
    best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    # Realiza predicciones
    predictions = best_model.predict(X_test)
    y_pred = np.argmax(predictions, axis=1)

    # Calcula métricas
    report = classification_report(y_test, y_pred, output_dict=True)

    # Registra los resultados en MLflow
    mlflow.log_param("best_params", best_params)
    mlflow.log_metric("best_score", best_score)
    mlflow.log_metric("accuracy", report['accuracy'])
    mlflow.log_metric("macro_f1_score", report['macro avg']['f1-score'])
    mlflow.log_metric("weighted_precision", report['weighted avg']['precision'])
    mlflow.log_metric("weighted_recall", report['weighted avg']['recall'])
    mlflow.log_metric("weighted_f1_score", report['weighted avg']['f1-score'])

AttributeError: module 'tensorflow.keras' has no attribute 'wrappers'

### Experimento metricas

In [12]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Supongamos que tienes tus datos en X y y, donde X son los comentarios y y son las etiquetas de quejas

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas para cada etiqueta
    precision = precision_score(y_test, predictions, average=None)
    recall = recall_score(y_test, predictions, average=None)
    f1 = f1_score(y_test, predictions, average=None)
    #roc_auc = roc_auc_score(y_test, predictions_proba, average=None, multi_class='ovr')

    # Registrar métricas para cada etiqueta
    for i in range(len(precision)):
        mlflow.log_metric(f"precision_label_{i}", precision[i])
        mlflow.log_metric(f"recall_label_{i}", recall[i])
        mlflow.log_metric(f"f1_label_{i}", f1[i])
        #mlflow.log_metric(f"roc_auc_label_{i}", roc_auc[i])
    
    # También puedes calcular métricas agregadas, como el promedio de todas las etiquetas
    mlflow.log_metric("precision_macro_avg", precision.mean())
    mlflow.log_metric("recall_macro_avg", recall.mean())
    mlflow.log_metric("f1_macro_avg", f1.mean())
    #mlflow.log_metric("roc_auc_macro_avg", roc_auc.mean())
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: U